In [19]:
!pip install tmdbv3api


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
from tmdbv3api import TMDb
from tmdbv3api import Movie
from tmdbv3api.exceptions import TMDbException
import random
import pandas as pd
import csv
tmdb=TMDb()
tmdb.api_key='9cf68f4c97c8f0cc6bb9646da389a808'
tmdb.language='en'
movie=Movie()

In [22]:
full_data = pd.read_csv('merged_data_mod_scores.csv', header=0, index_col=0)
full_data

,Running Time,Genres,IMDB Title,MPAA,Domestic Distributor,Domestic Opening,Earliest Release Date,IMDB Budget,TMDB Budget,Genre ID,...,Release Year,Merged Budget,Merged Revenue,Raw Star Scores,Unweighted Star Score,Simple Weight Star Score,Log Weight Star Score,Exponential Weight Star Score,director_scores,production_company_scores
0,1 hr 22 min,Documentary Sport,Into the Mind (2013),NaN,NaN,NaN,"January 31, 2014 (Spain)",0,0,[99],...,2013,100000,2241,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0.0,0.00000,0.000000,0.0,0.0
1,8 min,History Short,In Search of Lost Time (2013),NaN,NaN,NaN,"September 30, 2007 (South Korea)",0,0,[99],...,2013,100000,37,[],0,0.0,0.00000,0.000000,0.0,0.0
2,1 hr 49 min,Action Crime Drama Mystery Thriller,Broken City (2013),R,Twentieth Century Fox,8268908.0,"January 18, 2013 (Domestic, EMEA)",35000000,56000000,"[53, 80, 18]",...,2013,35000000,19701164,"[295, 158, 21, 0, 0, 0, 0, 0, 80, 0, 0, 0, 0, ...",554,470.0,459.47885,444.130435,67.0,360.0
3,1 hr 30 min,Drama,Redwood Highway (2013),PG-13,Monterey Media,30.0,"April 11, 2014 (Domestic)",0,0,[18],...,2013,100000,134049,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0.0,0.00000,0.000000,0.0,0.0
4,1 hr 46 min,Biography Drama History War,"Bless Me, Ultima (2012)",PG-13,Arenas Entertainment,470109.0,"February 22, 2013 (Domestic)",0,0,"[18, 10752]",...,2013,100000,1561962,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0.0,0.00000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9801,1 hr 44 min,Biography Documentary Sport,The Two Escobars (2010),NaN,All Rise Films,NaN,"August 27, 2010 (Domestic)",0,0,"[80, 99]",...,2010,100000,14457,[],0,0.0,0.00000,0.000000,NaN,NaN
9802,NaN,Horror,Takien: The Haunted Tree (2010),NaN,NaN,NaN,"June 17, 2010 (Thailand)",0,0,[27],...,2010,100000,117338,[],0,0.0,0.00000,0.000000,NaN,NaN
9803,1 hr 31 min,Documentary Music,Gypsy Spirit: Harri Stojka - Eine Reise (2010),NaN,NaN,NaN,"September 3, 2010 (Austria)",0,0,"[99, 10402]",...,2010,100000,84370,[],0,0.0,0.00000,0.000000,NaN,NaN
9804,1 hr 20 min,Drama Fantasy Horror,Evidence of a Haunting (2010),NaN,Nocturnal Features,16800.0,"August 26, 2011 (Domestic)",0,25000,[27],...,2010,100000,29904,[],0,0.0,0.00000,0.000000,NaN,NaN


Want to first get the list of TMDB IDs
then API call TMDB to get the US release dates if they exist
reformat those release dates in the same format we want
add that as column

In [23]:
tmdb_ids = full_data['TMDB ID']

us_release_dates = {}
for id in tmdb_ids:
    try:
        int_id = int(id)
        movie_info=movie.details(int_id)
        if movie_info:
            release_dates=movie_info.get('release_dates',{})
            results=release_dates.get('results',[])

            for result in results:
            # THIS (BELOW) IS THE LINE ADDED FOR US RELEASE DATE ONLY
                if result.get('iso_3166_1')=='US':
                    for release_date in result.get('release_dates',[]):
                        if release_date.get('type')==3:
                    #Write the movie ID to the list
                            us_release_dates[int_id] = release_date.get('release_date')
    except:
        pass
    

In [24]:
#us_release_df = pd.DataFrame.from_dict(us_release_dates_float)
us_release_df = pd.DataFrame({'TMDB ID': list(us_release_dates.keys()), 'US Release Date': list(us_release_dates.values())})
full_data_us = full_data.merge(us_release_df, 'left', left_on='TMDB ID', right_on='TMDB ID')

In [25]:
full_data_us.to_csv('merged_data_mod_scores_US.csv')